# Frozen Lake 4x4
[Link](https://github.com/hamedmokazemi/DeepQLearning_FrozenLake_1)

In [2]:
import gym
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from collections import deque

In [4]:
custom_map = [
    'FFFF',
    'FHFH',
    'FFFH',
    'HFFG'
]

env = gym.make("FrozenLake-v1", desc=custom_map, is_slippery=False)
train_episodes=4000
test_episodes=100
max_steps=300
state_size = env.observation_space.n
action_size = env.action_space.n
batch_size=32

In [7]:
learning_rate=0.001
epsilon=1
max_eps=1
min_eps=0.01
eps_decay = 0.001/3
gamma=0.9

In [9]:
class Agent:
    def __init__(self, state_size, action_size):
        self.memory = deque(maxlen=2500)
        self.learning_rate=learning_rate
        self.epsilon=epsilon
        self.max_eps=max_eps
        self.min_eps=min_eps
        self.eps_decay = eps_decay
        self.gamma=gamma
        self.state_size= state_size
        self.action_size= action_size
        self.epsilon_lst=[]
        self.model = self.buildmodel()

    def buildmodel(self):
        model=Sequential()
        model.add(Dense(10, input_dim=self.state_size, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def add_memory(self, new_state, reward, done, state, action):
        self.memory.append((new_state, reward, done, state, action))

    def action(self, state):
        if np.random.rand() > self.epsilon:
            return np.random.randint(0,4)
        return np.argmax(self.model.predict(state))

    def pred(self, state):
        return np.argmax(self.model.predict(state))

    def replay(self,batch_size):
        minibatch=random.sample(self.memory, batch_size)
        for new_state, reward, done, state, action in minibatch:
            target= reward
            if not done:
                target=reward + self.gamma* np.amax(self.model.predict(new_state))
            target_f= self.model.predict(state)
            target_f[0][action]= target
            self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.min_eps:
            self.epsilon=(self.max_eps - self.min_eps) * np.exp(-self.eps_decay*episode) + self.min_eps

        self.epsilon_lst.append(self.epsilon)

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [10]:
agent=Agent(state_size, action_size)
reward_lst=[]

In [11]:
for episode in range(train_episodes):
    state= env.reset()
    state_arr=np.zeros(state_size)
    state_arr[state] = 1
    state= np.reshape(state_arr, [1, state_size])
    reward = 0
    done = False
    for t in range(max_steps):
        # env.render()
        action = agent.action(state)
        new_state, reward, done, info = env.step(action)
        new_state_arr = np.zeros(state_size)
        new_state_arr[new_state] = 1
        new_state = np.reshape(new_state_arr, [1, state_size])
        agent.add_memory(new_state, reward, done, state, action)
        state= new_state

        if done:
            print(f'Episode: {episode:4}/{train_episodes} and step: {t:4}. Eps: {float(agent.epsilon):.2}, reward {reward}')
            break

    reward_lst.append(reward)

    if len(agent.memory)> batch_size:
        agent.replay(batch_size)

print(' Train mean % score= ', round(100*np.mean(reward_lst),1))

Episode:    0/4000 and step:    3. Eps: 1.0, reward 0.0
Episode:    1/4000 and step:    3. Eps: 1.0, reward 0.0
Episode:    2/4000 and step:    3. Eps: 1.0, reward 0.0
Episode:    3/4000 and step:    3. Eps: 1.0, reward 0.0
Episode:    4/4000 and step:    3. Eps: 1.0, reward 0.0
Episode:    5/4000 and step:    3. Eps: 1.0, reward 0.0
Episode:    6/4000 and step:    3. Eps: 1.0, reward 0.0
Episode:    7/4000 and step:    3. Eps: 1.0, reward 0.0
Episode:    8/4000 and step:    3. Eps: 1.0, reward 0.0
Episode:    9/4000 and step:    3. Eps: 1.0, reward 0.0
Episode:   10/4000 and step:    3. Eps: 1.0, reward 0.0
Episode:   11/4000 and step:    3. Eps: 1.0, reward 0.0
Episode:   12/4000 and step:    3. Eps: 1.0, reward 0.0
Episode:   13/4000 and step:    3. Eps: 1.0, reward 0.0
Episode:   14/4000 and step:    3. Eps: 1.0, reward 0.0
Episode:   15/4000 and step:   99. Eps: 1.0, reward 0.0
Episode:   16/4000 and step:    3. Eps: 1.0, reward 0.0
Episode:   17/4000 and step:    3. Eps: 0.99, re